# Parkinson's Mouse Tutorial - Taxonomy Assignment

Run this notebook in `qiime2-2021.11`.

Continuing the [pd-mouse tutorial](https://docs.qiime2.org/2021.11/tutorials/pd-mice/). Specifically the [Taxonomy](https://docs.qiime2.org/2021.11/tutorials/pd-mice/#taxonomic-classification), and [Phylogeny](https://docs.qiime2.org/2021.11/tutorials/pd-mice/#generating-a-phylogenetic-tree-for-diversity-analysis) steps. *Note we'll use a *de novo* [align-to-tree-mafft-fasttree ](https://docs.qiime2.org/2021.11/tutorials/phylogeny/#pipelines) step so we can run through this tutorial quicker.*

In [ ]:
from os import getcwd, listdir, chdir, mkdir
import qiime2 as q2

In [ ]:
getcwd()

In [ ]:
chdir('../processed')
getcwd()

## Download classifiers

We'll assign taxonomy using SILVA. Can obtain classifiers from the [Data Resource Page](https://docs.qiime2.org/2021.11/data-resources/).

In [ ]:
mkdir('silva-classifiers')

In [ ]:
! wget https://data.qiime2.org/2021.11/common/silva-138-99-515-806-nb-classifier.qza \
    -O ./silva-classifiers/silva-138-99-515-806-nb-classifier.qza

## Classify sequences / reads

In [ ]:
! qiime feature-classifier classify-sklearn \
    --i-reads ./dada2_rep_set.qza \
    --i-classifier ./silva-classifiers/silva-138-99-515-806-nb-classifier.qza \
    --p-n-jobs 2 \
    --o-classification ./taxonomy.qza

In [ ]:
# View list of classifications
! qiime metadata tabulate \
    --m-input-file ./taxonomy.qza \
    --o-visualization ./taxonomy.qzv

In [ ]:
q2.Visualization.load('taxonomy.qzv')

In [ ]:
# View a taxonomy barplot
! qiime taxa barplot \
    --i-table ./dada2_table.qza \
    --i-taxonomy ./taxonomy.qza \
    --m-metadata-file ./metadata.tsv \
    --o-visualization ./taxa_barplot.qzv

In [ ]:
q2.Visualization.load('taxa_barplot.qzv')

## Remove poorly classified reads

[Filtering Documentation](https://docs.qiime2.org/2020.11/tutorials/filtering/)

In [ ]:
! qiime taxa filter-table \
    --i-table ./dada2_table.qza \
    --i-taxonomy ./taxonomy.qza \
    --p-mode 'contains'  \
    --p-include 'p__' \
    --p-exclude 'p__;,Eukaryota,Chloroplast,Mitochondria' \
    --o-filtered-table ./table-no-ecmu.qza

In [ ]:
! qiime feature-table filter-seqs \
    --i-data ./dada2_rep_set.qza \
    --i-table ./table-no-ecmu.qza \
    --o-filtered-data rep_set-no-ecmu.qza

In [ ]:
! qiime tools export \
    --input-path rep_set-no-ecmu.qza \
    --output-path rep_set-no-ecmu-export

In [ ]:
# View a taxonomy barplot
! qiime taxa barplot \
    --i-table ./table-no-ecmu.qza \
    --i-taxonomy ./taxonomy.qza \
    --m-metadata-file ./metadata.tsv \
    --o-visualization ./table-no-ecmu-taxa-barplot.qzv

In [ ]:
q2.Visualization.load('table-no-ecmu-taxa-barplot.qzv')

#### krona plot

In [ ]:
! qiime krona collapse-and-plot \
    --i-table ./table-no-ecmu.qza \
    --i-taxonomy ./taxonomy.qza \
    --o-krona-plot ./table-no-ecmu-taxa-krona.qzv

##  Other QA / QC Operations

See [q2-quality-control tutorial](https://docs.qiime2.org/2021.11/tutorials/quality-control/).

In [ ]:
mkdir('references')

In [ ]:
# download pre-made SILVA refrence
! wget https://data.qiime2.org/2021.11/common/silva-138-99-seqs-515-806.qza \
    -O ./references/silva-138-99-seqs-515-806.qza

In [ ]:
# remove poor quality sequence that do not have a decent match to our curated reference database.
! qiime quality-control exclude-seqs \
    --i-query-sequences ./rep_set-no-ecmu.qza \
    --i-reference-sequences ./references/silva-138-99-seqs-515-806.qza \
    --p-method vsearch \
    --p-perc-identity 0.90 \
    --p-perc-query-aligned 0.90 \
    --o-sequence-hits ./hits.qza \
    --o-sequence-misses ./misses.qza

In [ ]:
# filter table to match filtered sequence file
! qiime feature-table filter-features \
    --i-table ./table-no-ecmu.qza \
    --m-metadata-file ./hits.qza \
    --o-filtered-table ./table-no-ecmu-hits.qza

#### Given that we filtered our data again, you may want to re-generate the taxonomy plots. Use the prior taxonomy visualization commands above as a guid and run them below, with the new table:

In [ ]:
# updated taxonomy barplot
! qiime taxa barplot \
    --i-table ./table-no-ecmu-hits.qza \
    --i-taxonomy ./taxonomy.qza \
    --m-metadata-file ./metadata.tsv \
    --o-visualization ./table-no-ecmu-hits-taxa-barplot.qzv

In [ ]:
# updated krona plot
! qiime krona collapse-and-plot \
    --i-table ./table-no-ecmu-hits.qza \
    --i-taxonomy ./taxonomy.qza \
    --o-krona-plot ./table-no-ecmu-hits-taxa-krona.qzv

#### krona collapse by group 

In [ ]:
! qiime feature-table group \
    --i-table ./table-no-ecmu-hits.qza \
    --p-axis sample \
    --m-metadata-file ./metadata.tsv \
    --m-metadata-column donor \
    --p-mode 'mean-ceiling' \
    --o-grouped-table ./table-no-ecmu-hits-donor.qza

In [ ]:
! qiime krona collapse-and-plot \
    --i-table ./table-no-ecmu-hits-donor.qza \
    --i-taxonomy ./taxonomy.qza \
    --o-krona-plot ./table-no-ecmu-hits-donor-taxa-krona.qzv

## Construct phylogeny

See the [Inferring Phylogenies tutorial](https://docs.qiime2.org/2021.11/tutorials/phylogeny/) for more information.

We'll run [FastTree](https://docs.qiime2.org/2021.11/tutorials/phylogeny/#fasttree) to be quick, though I'd recomend [iqtree](https://docs.qiime2.org/2021.11/tutorials/phylogeny/#iqtree) or [fragment-insertion](https://library.qiime2.org/plugins/q2-fragment-insertion/16/).

We'll be using the [align-to-tree-mafft-fasttree](https://docs.qiime2.org/2021.11/tutorials/phylogeny/#pipelines) pipeline.

### *de novo phylogeny*

View with [iTOL](https://itol.embl.de/) or [Empress](https://github.com/biocore/empress).

In [ ]:
# pipeline: alignment through phylogeny
! qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences ./hits.qza \
    --output-dir ./mafft-fasttree-output \
    --verbose

### Another phylogenetic approach: Fragment Insertion

## [Empress](https://github.com/biocore/empress)

In [ ]:
!qiime empress tree-plot \
    --i-tree ./mafft-fasttree-output/rooted_tree.qza \
    --m-feature-metadata-file ./taxonomy.qza \
    --o-visualization ./tree-viz.qzv

In [ ]:
q2.Visualization.load('./tree-viz.qzv')

In [ ]:
! qiime empress community-plot \
    --i-tree ./mafft-fasttree-output/rooted_tree.qza \
    --i-feature-table ./table-no-ecmu-hits.qza \
    --m-sample-metadata-file ./metadata.tsv \
    --m-feature-metadata-file ./taxonomy.qza \
    --o-visualization tree-tax-table.qzv

In [ ]:
q2.Visualization.load('./tree-tax-table.qzv')